In [1]:
"""
------------------------------------------------------------------------------
Libraries
------------------------------------------------------------------------------
"""

import requests
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point

import time

import re
from datetime import datetime

In [2]:
xmasData = pd.read_csv("Weihnachtsmärkte25-roh.csv", sep=";")

print(xmasData)

     id                                          bezirk  \
0   109                            Tempelhof-Schöneberg   
1    79  Lichtenberg, Mitte, Charlottenburg-Wilmersdorf   
2   118                            Tempelhof-Schöneberg   
3    82                            Tempelhof-Schöneberg   
4     7                             Marzahn-Hellersdorf   
5    19                      Charlottenburg-Wilmersdorf   
6    85                      Charlottenburg-Wilmersdorf   
7    64                      Charlottenburg-Wilmersdorf   
8   100                             Marzahn-Hellersdorf   
9    58                      Charlottenburg-Wilmersdorf   
10   88                                     Brandenburg   
11   67                                         Wedding   
12  130                                     Brandenburg   
13   61                                  Friedrichshain   
14   94                                           Mitte   
15   49                             Steglitz-Zehlendorf 

In [3]:
df = xmasData.copy()

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

df.head()

,id,bezirk,name,strasse,plz_ort,von,bis,veranstalter,oeffnungszeiten,email,www,barrierefreiheit,bemerkungen
0,109,Tempelhof-Schöneberg,Winter am THF,THF TOWER - Flughafen Tempelhof,12101.0,NaN,NaN,Flughafen Tempelhof (Tempelhof Projekt GmbH),2. Adventswochenende: 05. bis 07.12.2025 \n3. Adventswochenende: 12. bis 14.12.2025 \n\nFreitags 15:00 – 20:00 Uhr \nSamstags und Sonntags 12:00 – 20:00 Uhr,kommunikation@thf-berlin.de,https://www.thf-berlin.de/winter-am-thf,ja,"Die Alte Feuerwache ist barrierefrei von außen zugänglich, die 6. Etage des THF TOWERs ist barrierefrei über einen Fahrstuhl zugänglich. Wintermarkt in der Alten Feuerwache am THF TOWER mit: - Foodtrucks &amp; Glühwein - Handgemachten Designobjekten und Geschenkideen - Weihnachtlicher Musik - Basteln &amp; Weihnachtsfilme für Kinder (6. Etage des THF TOWERs)\n Außerdem findet am 14.12. ein großes Weihnachtssingen in der ehemaligen Haupthalle des Flughafen Tempelhof statt!"
1,79,"Lichtenberg, Mitte, Charlottenburg-Wilmersdorf","Weihnachtsmarkt Verkauf von Trüffel Pasta, Dessert","Lichtenberg, Alexanderplatz, Potsdamer Platz oder Kudamm (nach Möglichkeit)",NaN,01.11.2025,31.12.2025,Aposto Gaststätten GmbH,11:00 bis 23:00 Uhr,lumi-sadiki@hotmail.de,https://aposto-berlin.de/,Ja,NaN
2,118,Tempelhof-Schöneberg,LGBT*Winterdays und Christmas Avenue Nollendorfplatz,Nollendorfplatz unter der Hochbahn,10777.0,05.11.2025,23.12.2025,Rutwiess Events Berlin & LGBT*Aktionsgemeinschaft Berlin,LGBTQIA* Winterdays:,Info@Rutwiess-events.de,https://www.christmas-avenue.berlin/,nein,"auf dem Markt sind Holzhecksel gestreut. Sollten gehandicapte Besucher:innen hier Hilfe benötigen, spechen sie gerne unsere Mitarbeiter an Christmas Avenue & LGBTQIA Winterdays 2025: Glitzer, Beats und queere Liebe am Nollendorfplatz* Ab dem 5. November starten die LGBTQIA Winterdays*, ab dem 24. November öffnet die CHRISTMAS AVENUE – Berlins einziger queerer Weihnachtsmarkt – wieder ihre Pforten. Bis 23. Dezember wird der Nollendorfplatz zum funkelnden Treffpunkt für Community, Freund:innen und Familien. Wenn draußen das Grau regiert, leuchtet unter der Hochbahn ein Ort voller Glitzer, Wärme und Miteinander: Die Christmas Avenue wird auch 2025 wieder das weihnachtliche Wohnzimmer der queeren Szene.\n Zum Start lädt Drag-Queen Margot Schlönzke am 5. November zum legendären Drag-Bingo. Danach sorgen Live-Musik, Drag-Shows, Karaoke und die heißesten DJs der Szene für Partylaune – von House bis Pop. Berlins bekannteste Drag-Performerinnen übernehmen Moderation und Bühne. Dazu gibt’s kulinarische Highlights, Kunst, Design & Manufaktur-Stände, wettergeschützte Gemütlichkeit unter der Hochbahn – und jede Menge queere Sichtbarkeit: Vereine und Initiativen präsentieren ihre Arbeit, sammeln Spenden und laden zum Austausch ein. Für Gruppen gibt es eine Partyhütte zum Mieten. \n Wöchentliche Specials wie Happy Hour (Mo), After Work (Do) und Aperol o’Clock (So) runden das Programm ab.\n Fotos können auf unsere Homepage zur freien Verfügung runtergeladen werden."
3,82,Tempelhof-Schöneberg,30. Weihnachtsmarkt auf Lehmanns Bauernhof,Lehmanns Bauernhof \nAlt-Marienfelde 35-37,12277.0,05.12.2025 \n12.12.2025,07.12.2025 \n14.12.2025,Lehmanns Bauernhof E&E Trade Event UG (haftungsbeschränkt) & Co. KG,Freitag: 14 - 20 Uhr \nSamstag: 12 - 20 Uhr \nSonntag: 12 - 20 Uhr,karstenemail@gmx.de,https://www.lehmannsbauernhof.de,Ja,"ebenerdiges Gelände, teilweise Kopfsteinpflaster, mit gut zugänglichen Innenräumen. Auf einem denkmalgeschützten Vierseitenhof, direkt gegenüber der Dorfkirche an der romantischen Dorfaue gelegen, findet zum 30. Mal ein familiär geprägter Weihnachtsmarkt statt. Der Fokus liegt auf Kunsthandwerk und außergewöhnlichen Geschenkidee, ergänzt durch weihnachtliche Gaumengenüsse. Ein Bühnenprogramm, eine Märchenerzählerin sowie Ponyreiten und die Tiere des Hofes runden das Erlebnis bei freiem Eintritt ab."
4,7,Marzahn-Hellersdorf,Weihnachtsmarkt Alt-Kaulsdorf,Alt-Kaulsdorf Dorfstrasse,12621.0,06.12.2025,NaN,Heimatv

In [4]:
#Zeilenumbrüche entfernen, damit csv richtig generiert wird
import pandas as pd
obj_cols = xmasData.select_dtypes(include=["object", "string"]).columns

xmasData = xmasData.replace(r"[\r\n]+", " | ", regex=True)


In [5]:
# Prüfen ob noch Zeilenumbrüche drin sind
counts = xmasData.apply(
    lambda s: s.astype(str).str.contains(r"[\r\n]", na=False).sum()
).sort_values(ascending=False)

for col, n in counts.items():
    print(f"{col}: {n} Zeilen mit Umbruch")

id: 0 Zeilen mit Umbruch
bezirk: 0 Zeilen mit Umbruch
name: 0 Zeilen mit Umbruch
strasse: 0 Zeilen mit Umbruch
plz_ort: 0 Zeilen mit Umbruch
von: 0 Zeilen mit Umbruch
bis: 0 Zeilen mit Umbruch
veranstalter: 0 Zeilen mit Umbruch
oeffnungszeiten: 0 Zeilen mit Umbruch
email: 0 Zeilen mit Umbruch
www: 0 Zeilen mit Umbruch
barrierefreiheit: 0 Zeilen mit Umbruch
bemerkungen: 0 Zeilen mit Umbruch


In [6]:
xmasData.head(30)

,id,bezirk,name,strasse,plz_ort,von,bis,veranstalter,oeffnungszeiten,email,www,barrierefreiheit,bemerkungen
0,109,Tempelhof-Schöneberg,Winter am THF,THF TOWER - Flughafen Tempelhof,12101.0,NaN,NaN,Flughafen Tempelhof (Tempelhof Projekt GmbH),2. Adventswochenende: 05. bis 07.12.2025 | 3. Adventswochenende: 12. bis 14.12.2025 | Freitags 15:00 – 20:00 Uhr | Samstags und Sonntags 12:00 – 20:00 Uhr,kommunikation@thf-berlin.de,https://www.thf-berlin.de/winter-am-thf,ja,"Die Alte Feuerwache ist barrierefrei von außen zugänglich, die 6. Etage des THF TOWERs ist barrierefrei über einen Fahrstuhl zugänglich. Wintermarkt in der Alten Feuerwache am THF TOWER mit: - Foodtrucks &amp; Glühwein - Handgemachten Designobjekten und Geschenkideen - Weihnachtlicher Musik - Basteln &amp; Weihnachtsfilme für Kinder (6. Etage des THF TOWERs) | Außerdem findet am 14.12. ein großes Weihnachtssingen in der ehemaligen Haupthalle des Flughafen Tempelhof statt!"
1,79,"Lichtenberg, Mitte, Charlottenburg-Wilmersdorf","Weihnachtsmarkt Verkauf von Trüffel Pasta, Dessert","Lichtenberg, Alexanderplatz, Potsdamer Platz oder Kudamm (nach Möglichkeit)",NaN,01.11.2025,31.12.2025,Aposto Gaststätten GmbH,11:00 bis 23:00 Uhr,lumi-sadiki@hotmail.de,https://aposto-berlin.de/,Ja,NaN
2,118,Tempelhof-Schöneberg,LGBT*Winterdays und Christmas Avenue Nollendorfplatz,Nollendorfplatz unter der Hochbahn,10777.0,05.11.2025,23.12.2025,Rutwiess Events Berlin & LGBT*Aktionsgemeinschaft Berlin,LGBTQIA* Winterdays:,Info@Rutwiess-events.de,https://www.christmas-avenue.berlin/,nein,"auf dem Markt sind Holzhecksel gestreut. Sollten gehandicapte Besucher:innen hier Hilfe benötigen, spechen sie gerne unsere Mitarbeiter an Christmas Avenue & LGBTQIA Winterdays 2025: Glitzer, Beats und queere Liebe am Nollendorfplatz* Ab dem 5. November starten die LGBTQIA Winterdays*, ab dem 24. November öffnet die CHRISTMAS AVENUE – Berlins einziger queerer Weihnachtsmarkt – wieder ihre Pforten. Bis 23. Dezember wird der Nollendorfplatz zum funkelnden Treffpunkt für Community, Freund:innen und Familien. Wenn draußen das Grau regiert, leuchtet unter der Hochbahn ein Ort voller Glitzer, Wärme und Miteinander: Die Christmas Avenue wird auch 2025 wieder das weihnachtliche Wohnzimmer der queeren Szene. | Zum Start lädt Drag-Queen Margot Schlönzke am 5. November zum legendären Drag-Bingo. Danach sorgen Live-Musik, Drag-Shows, Karaoke und die heißesten DJs der Szene für Partylaune – von House bis Pop. Berlins bekannteste Drag-Performerinnen übernehmen Moderation und Bühne. Dazu gibt’s kulinarische Highlights, Kunst, Design & Manufaktur-Stände, wettergeschützte Gemütlichkeit unter der Hochbahn – und jede Menge queere Sichtbarkeit: Vereine und Initiativen präsentieren ihre Arbeit, sammeln Spenden und laden zum Austausch ein. Für Gruppen gibt es eine Partyhütte zum Mieten. | Wöchentliche Specials wie Happy Hour (Mo), After Work (Do) und Aperol o’Clock (So) runden das Programm ab. | Fotos können auf unsere Homepage zur freien Verfügung runtergeladen werden."
3,82,Tempelhof-Schöneberg,30. Weihnachtsmarkt auf Lehmanns Bauernhof,Lehmanns Bauernhof | Alt-Marienfelde 35-37,12277.0,05.12.2025 | 12.12.2025,07.12.2025 | 14.12.2025,Lehmanns Bauernhof E&E Trade Event UG (haftungsbeschränkt) & Co. KG,Freitag: 14 - 20 Uhr | Samstag: 12 - 20 Uhr | Sonntag: 12 - 20 Uhr,karstenemail@gmx.de,https://www.lehmannsbauernhof.de,Ja,"ebenerdiges Gelände, teilweise Kopfsteinpflaster, mit gut zugänglichen Innenräumen. Auf einem denkmalgeschützten Vierseitenhof, direkt gegenüber der Dorfkirche an der romantischen Dorfaue gelegen, findet zum 30. Mal ein familiär geprägter Weihnachtsmarkt statt. Der Fokus liegt auf Kunsthandwerk und außergewöhnlichen Geschenkidee, ergänzt durch weihnachtliche Gaumengenüsse. Ein Bühnenprogramm, eine Märchenerzählerin sowie Ponyreiten und die Tiere des Hofes runden das Erlebnis bei freiem Eintritt ab."
4,7,Marzahn-Hellersdorf,Weihnachtsmarkt Alt-Kaulsdorf,Alt-Kaulsdorf Dorfstrasse,12621.0,06.12.2025,NaN,Heimatvere

In [7]:
#csv

xmasData.to_csv("markets_dataProcessing_firstcheck.csv", index= False, sep=";")
